In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image

In [ ]:
data_test = []
path = 'Dataset/Test'
l = os.listdir(path)
for i in l:
    imge = Image.open(path+'/'+str(i))
    imge = imge.resize((60,60))
    imge = np.array(imge)
    data_test.append(imge)
data_test = np.array(data_test)
data_test = data_test/255

In [ ]:
tot_noise = 100
Z=[]

for i in range(tot_noise):
        
    noise = np.random.randn(108)
    Z.append(noise)

Z=np.array(Z)
Z.shape

In [ ]:
descriminator = tf.keras.Sequential()
descriminator.add(tf.keras.layers.Conv2D(100,(3,3),activation=tf.keras.layers.LeakyReLU(),input_shape=(60,60,3),padding="same"))
descriminator.add(tf.keras.layers.Conv2D(50,(3,3),activation=tf.keras.layers.LeakyReLU()))
# descriminator.add(tf.keras.layers.MaxPooling2D((2,2)))
descriminator.add(tf.keras.layers.Conv2D(50,(3,3),activation=tf.keras.layers.LeakyReLU()))
# descriminator.add(tf.keras.layers.MaxPooling2D((2,2)))
descriminator.add(tf.keras.layers.Conv2D(20,(3,3),activation=tf.keras.layers.LeakyReLU()))
# descriminator.add(tf.keras.layers.MaxPooling2D((2,2)))
descriminator.add(tf.keras.layers.Flatten())
# descriminator.add(tf.keras.layers.Dense(100,activation="sigmoid"))
descriminator.add(tf.keras.layers.Dense(1,activation="sigmoid"))


In [ ]:
descriminator.summary()

In [ ]:
generator = tf.keras.Sequential()
# generator.add(tf.keras.layers.Flatten(input_shape=(108,)))
generator.add(tf.keras.layers.Reshape((3,6,6),input_shape = (108,)))
generator.add(tf.keras.layers.Conv2DTranspose(20,(2,2),activation=tf.keras.layers.LeakyReLU()))
generator.add(tf.keras.layers.Conv2DTranspose(50,(3,3),activation=tf.keras.layers.LeakyReLU()))
generator.add(tf.keras.layers.Conv2DTranspose(100,(4,4),activation=tf.keras.layers.LeakyReLU()))
# generator.add(tf.keras.layers.Flatten())
# generator.add(tf.keras.layers.Dense(10800,activation="sigmoid"))
generator.add(tf.keras.layers.Reshape((60,60,3)))


In [ ]:
generator.summary()

In [ ]:
generator.trainable_variables

In [ ]:
i= np.random.choice(range(100))
z=Z[i,:]
print(z.shape)
z = z.reshape(-1,108,)


In [ ]:
X = data_test[0].reshape(-1,60,60,3)
X = tf.constant(X)
fake_X = tf.constant(generator(z))

In [ ]:
plt.imshow(X[0])

In [ ]:
plt.imshow(fake_X[0])

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(0.1)
discriminator_optimizer = tf.keras.optimizers.Adam(0.1)

In [ ]:
for epochs in range(50):    
    with tf.GradientTape() as disc_tape:
        descriminator.trainable=True
        generator.trainable = False
        real_output = descriminator(X)
        fake_output = descriminator(generator(z))
        
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_discriminator = disc_tape.gradient(disc_loss, descriminator.trainable_variables)
    print(gradients_of_discriminator)
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, descriminator.trainable_variables))
    with tf.GradientTape() as gen_tape:
        descriminator.trainable = False
        generator.trainable = True       
        fake_output = descriminator(generator(z))
        gen_loss = generator_loss(fake_output)
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    print("--------------------------")
    print(gradients_of_generator)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))


In [ ]:
plt.imshow(generator(z)[0])